In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from data_utils.data import CatsAndDogs
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F
from models.classifier import ClassifierWithCustomBatchNorm

In [2]:
BATCH_SIZE = 128
ETA = 1e-3
EPOCHS = 30

In [3]:
def compute_accuracy(ps, labels):
    equality = (torch.round(ps) == labels).float()
    acc = equality.mean()
    return acc

In [4]:
def train_step(context, x, y):
    model = context["model"]
    optimizer = context["optimizer"]
    
    optimizer.zero_grad()
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [5]:
def eval_step(context, x, y):
    model = context["model"]
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [6]:
def train():
    trainset = CatsAndDogs(mode="train")
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = CatsAndDogs(mode="test")
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    model = ClassifierWithCustomBatchNorm().cuda()
    optimizer = optim.Adam(model.parameters())
    
    context = {
        "model": model,
        "optimizer": optimizer
    }
    
    for e in range(EPOCHS):
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            loss, acc = train_step(context, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            with torch.no_grad():
                loss, acc = eval_step(context, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [7]:
train()

Epochs 1/30
Train loss: 0.71459579, train acc: 0.6029
Test loss: 0.77248961, test acc: 0.5161
Epochs 2/30
Train loss: 0.52223337, train acc: 0.8529
Test loss: 0.74346316, test acc: 0.6129
Epochs 3/30
Train loss: 0.45260517, train acc: 0.8503
Test loss: 0.71234030, test acc: 0.6452
Epochs 4/30
Train loss: 0.46945110, train acc: 0.8242
Test loss: 0.68783784, test acc: 0.6935
Epochs 5/30
Train loss: 0.33509566, train acc: 0.9232
Test loss: 0.65654331, test acc: 0.6290
Epochs 6/30
Train loss: 0.37468523, train acc: 0.8854
Test loss: 0.61289358, test acc: 0.6452
Epochs 7/30
Train loss: 0.30591156, train acc: 0.9349
Test loss: 0.59025389, test acc: 0.6935
Epochs 8/30
Train loss: 0.27236858, train acc: 0.9349
Test loss: 0.58787251, test acc: 0.7097
Epochs 9/30
Train loss: 0.26261911, train acc: 0.9427
Test loss: 0.58092743, test acc: 0.7097
Epochs 10/30
Train loss: 0.24284999, train acc: 0.9583
Test loss: 0.58251226, test acc: 0.6452
Epochs 11/30
Train loss: 0.19186089, train acc: 1.0000
Test